# Video Game Recomendation System

## Business understanding

## Data understanding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Dataset from kaggle on [here](https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset?select=amazon_reviews_us_Digital_Video_Games_v1_00.tsv)

In [2]:
df_raw = pd.read_csv('./data/Digital_Video_Games.tsv', sep='\t', header=0, error_bad_lines=False)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144724 entries, 0 to 144723
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        144724 non-null  object
 1   customer_id        144724 non-null  int64 
 2   review_id          144724 non-null  object
 3   product_id         144724 non-null  object
 4   product_parent     144724 non-null  int64 
 5   product_title      144724 non-null  object
 6   product_category   144724 non-null  object
 7   star_rating        144724 non-null  int64 
 8   helpful_votes      144724 non-null  int64 
 9   total_votes        144724 non-null  int64 
 10  vine               144724 non-null  object
 11  verified_purchase  144724 non-null  object
 12  review_headline    144722 non-null  object
 13  review_body        144722 non-null  object
 14  review_date        144721 non-null  object
dtypes: int64(5), object(10)
memory usage: 16.6+ MB


### Columns
- **marketplace**: 2 letter country code of the marketplace where the review was written.
- **customer_id**: Random identifier that can be used to aggregate reviews written by a single author.
- **review_id**: The unique ID of the review.
- **productid**: The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same productid.
- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.
- **product_title**: Title of the product.
- **product_category**: Broad product category that can be used to group reviews (also used to group the dataset into coherent parts).
- **star_rating**: The 1-5 star rating of the review.
- **helpful_votes**: Number of helpful votes.
- **total_votes**: Number of total votes the review received.
- **vine**: Review was written as part of the Vine program.
- **verified_purchase**: The review is on a verified purchase.
- **review_headline**: The title of the review.
- **review_body**: The review text.
- **review_date**: The date the review was written.

In [3]:
df_raw['marketplace'].unique()

array(['US'], dtype=object)

All reviews in dataset are from US customers

In [4]:
df_raw['customer_id'].nunique()

112891

112891 different customers 

In [5]:
df_raw['review_id'].nunique()

144724

Seems like there are no duplicate reviews in the set

In [6]:
df_raw['product_id'].nunique()

7939

7939 different products, but not all video games, found "Playstation Plus Subscription" in row 3, so all digitally sold products related video games, like subscriptions, seem to be included in the data set, will further look for other kind of product other than games and drop those.

In [7]:
df_raw['product_parent'].nunique()

7755

- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.

seems like just another id but we have about 200 less data points in this column than product id

In [8]:
df_raw['product_title'].nunique()

6946

If i have 6946 different titles, i would expect the same amount of product_id, wonder if by droping everything that is not a game, like substriptions, would fix this discrepancy.

In [9]:
df_raw.iloc[3]['product_title']

'Playstation Plus Subscription'

It doesnt specify if the subscription is for 1, 3 or 12 months

<br>

Below i used key words to look for any products that is not a game
e.g
- Subscription
- Month
- Membership
- Card
- Network



In [10]:
def look_keyword(strg):
    content = df_raw[df_raw['product_title'].str.contains(strg)]
    print(f'Number of reviews for "{strg}" items: {len(content)} \n')
    
look_keyword("Subscription")    
look_keyword("Month")
look_keyword("Membership")
look_keyword("Card")
look_keyword("Network")

Number of reviews for "Subscription" items: 12137 

Number of reviews for "Month" items: 630 

Number of reviews for "Membership" items: 275 

Number of reviews for "Card" items: 21173 

Number of reviews for "Network" items: 13613 



In [11]:
df_raw['star_rating'].unique()
#games were rated in a scale of 1 to 5 stars

array([2, 5, 4, 1, 3])

In [12]:
df_raw['review_date'] = pd.to_datetime(df_raw['review_date'])
print(f'All reviews date back from {df_raw.review_date.min().year} to {df_raw.review_date.max().year}')

All reviews date back from 2006 to 2015


## Data Preparation

In [13]:
#Droping all rows that have strings: Subscription, Month, Membership,Network on the product_title column
df_raw_noSub = df_raw[df_raw["product_title"].str.contains("Subscription|Month|month|Membership|Network|Credit|Season Pass|Xbox Music Pass|Virtual Currency") == False]


Season Pass, Xbox Music Pass and Virtual Currency are some of the other kind of items on the dataset, i simply added them on the line above as i kept finding more during the process

For titles that have string 'Card' in the title, i have to be more careful since there are some game that include that on their name

In [14]:
Sub_Card2 = df_raw_noSub[df_raw_noSub['product_title'].str.contains('Card')]
Sub_Card2['product_title'].unique()

array(['Xbox Live Gift Card', 'Xbox 360 Live Points Card',
       'Grand Theft Auto V Cash Cards', 'Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       'Xbox $5 Gift Card - Xbox 360 Digital Code',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Xbox $15 Gift Card (Call of Duty Ghosts:\xa0Onslaught DLC) - Xbox 360 Digital Code',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]',
       '1-Year PS Plus + $10 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Five Card Deluxe [Download]',
       '1-Year PS Plus + $50 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Xbox Live $6 Gift Card - Xbox 360 Digital Code',
       '1-Year PS Plus + $

First i will drop rows where 'Gift Card' is found, then i will proceed with other common combinations of words like 'Xbox Live' and Live Points Card

In [15]:
df_raw_noSub2 = df_raw_noSub[df_raw_noSub["product_title"].str.contains("Gift Card|Xbox Live|Live Points Card|Xbox Live Gift Card|Grand Theft Auto V Cash Cards") == False]

Sub_Card3 = df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('Card')]
Sub_Card3['product_title'].unique()

array(['Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]', 'Five Card Deluxe [Download]',
       "King's Collection: 6 Classic Card Games",
       '8 Card Game Pack [Download]', '5 Realms of Cards [Download]',
       '5 Card Slingo [Download]', 'Card Crazy! [Download]',
       'Rift 30 Day Game Time Card [Online Game Code]',
       'Bicylce Family Card Games [Download]',
       'Rift 60 Day Game Time Card [Online Game Code]',
       'High Stakes Poker: Connelly Card Club [Online Game Code]',
       'Strange Cases: The Tarot Card Mystery [Download]'], dt

Seems like all items left that include "card" in the title are actual games

<br>

Lets look for DLCs

In [16]:
DLC = df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('DLC')]
print(f"Total number of DLCs: {DLC['product_title'].nunique()}")
print(f'Total number of DLCs reviews:{len(DLC)}')

Total number of DLCs: 212
Total number of DLCs reviews:1698


We have 212 different DLCs, and 1698 reviews, an original copy of the first game is required to be able to run a DLC, I will drop these too.

In [17]:
df_raw_noSub_no_DLC = df_raw_noSub2[df_raw_noSub2["product_title"].str.contains("DLC|Pack") == False]

Quantity for product id and product title still doesn't match

<br>

Some games include a substring next to the title, it describes the way buyers could access the game, for example:

`18 Wheels of Steel American Long Haul [Download]`

`18 Wheels of Steel American Long Haul [Online Game Code]`

Definitely the same game, just different "delivery" method.


In [18]:
df_raw_noSub_no_DLC[df_raw_noSub_no_DLC['product_title'].str.contains('18 Wheels of Steel American Long Haul')].head(3)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
39224,US,36420328,R2LOZS1XJTWZNN,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,ok,2014-10-15
50242,US,17845931,R2TJLHZ9X7EG83,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,Like driving trucks.,2014-07-14
74152,US,21374226,R388BHLY4GIU5E,B00CLVZGPK,775610170,18 Wheels of Steel American Long Haul [Online ...,Digital_Video_Games,1,1,1,N,Y,"Doesn't work, no manufacturer support",The code I got does not work. Amazon referred ...,2013-12-29



<br>

The `[Download]` edition of the game, on the second row above, has a different `product_id` than its `[Online Game Code]` edition (on the 3rd row above), it's the same game but taged with different ID, `product_parent` number is also different

<br>

I find it easier to modify the name than the id number, i need to group those reviews togeter, the delivery method doesnt change the software performance, so the review for a specific version is still relevant for a different one.

<br>

At first sight, `[Instant Access]`, `[Download]`, `[Digital Code]`, `[Game Connect]` and `[Online Game Code]` seem to be the most frequent tags at the end of the games titles, if i delete those out of the title name, i could use them for the rec system instead of the id, which was my first option

In [19]:
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['product_title'].str.replace(" \[Instant Access\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Download\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Game Connect\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Online Game Code\]", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Digital Code\]", "")


<ipython-input-19-de3b04f97402>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['product_title'].str.replace(" \[Instant Access\]", "")
<ipython-input-19-de3b04f97402>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" \[Download\]", "")
<ipython-input-19-de3b04f97402>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

The first title (on index 0) has 'Xbox One Digital Code' in its name, will check if there are more games with this same substring, or even for a different platform other than xbox

In [20]:
#below i deleted the susbstring using same method as before
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox One Digital Code", "")

#found it for the 360 console too
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox 360 Digital Code", "")

<ipython-input-20-00b8bc71ca71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox One Digital Code", "")
<ipython-input-20-00b8bc71ca71>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - Xbox 360 Digital Code", "")


In [21]:
#Now will look for something similar in a different platform
df_raw_noSub_no_DLC[df_raw_noSub_no_DLC['product_title'].str.contains('PS4')]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,clean_title
57,US,9095823,R2642823GCWLND,B011WI7U0G,864854550,Batman: Arkham Knight: A Matter Of Family - PS...,Digital_Video_Games,3,1,1,N,Y,Three Stars,It's easily better than the Harley Quinn dlc a...,2015-08-31,Batman: Arkham Knight: A Matter Of Family - PS4
76,US,51688000,R34Y0NX6RHTWH1,B012PRO97A,56796613,Sword Art Online Re: Hollow Fragment - PS4 [Di...,Digital_Video_Games,5,1,2,N,Y,Great gift option,Received code in seconds. No issues with the ...,2015-08-31,Sword Art Online Re: Hollow Fragment - PS4
81,US,17746031,R2BYOTSDJJY65O,B00DRKJBC8,29664234,Final Fantasy VII - PS4 [Digital Code],Digital_Video_Games,4,0,0,N,Y,An amazing game,"First of all, I never played the original Play...",2015-08-31,Final Fantasy VII - PS4
97,US,28034012,R2GFH6IGSW33MP,B00GMPJKDA,806044015,Trine 2: Complete Story - PS4 [Digital Code],Digital_Video_Games,4,1,1,N,Y,Four Stars,"Awesome brain candy, visually beautiful and in...",2015-08-31,Trine 2: Complete Story - PS4
122,US,3978884,R1GQER9Z4SUO2A,B00JAPIV84,841717561,Dead Nation Apocalypse Edition - PS4 [Digital ...,Digital_Video_Games,5,1,1,N,Y,Five Stars,very good,2015-08-30,Dead Nation Apocalypse Edition - PS4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87719,US,44206257,R28CULMVKI3STF,B00DRKJBC8,29664234,Final Fantasy VII - PS4 [Digital Code],Digital_Video_Games,4,2,5,N,Y,I can't believe I found it,I have been a huge fan of FF since the first o...,2013-08-20,Final Fantasy VII - PS4
88499,US,18170799,R1WK8YSYXTH0ZW,B00DRKJBC8,29664234,Final Fantasy VII - PS4 [Digital Code],Digital_Video_Games,5,1,1,N,Y,Awesome,"I used to play this when I was younger, and is...",2013-08-13,Final Fantasy VII - PS4
89606,US,30522361,R2JM2FL74BQWUE,B00DRKJBC8,29664234,Final Fantasy VII - PS4 [Digital Code],Digital_Video_Games,3,1,19,N,Y,Best RPG ever!,The game is amazing and gets 5 stars. The down...,2013-08-03,Final Fantasy VII - PS4
90688,US,17058574,R1K7X74SN65PV4,B00DRKJBC8,29664234,Final Fantasy VII - PS4 [Digital Code],Digital_Video_Games,4,40,47,N,N,(Almost) Exactly as you remember it,...For better or worse.<br /><br />This is in ...,2013-07-24,Final Fantasy VII - PS4


In [22]:
#found ' - PS4', ' - PS3', ' - PS Vita / PS4 / PS3', ' - PS Vita'
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita / PS4 / PS3", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS3", "")
df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS4", "")

<ipython-input-22-4559ff269911>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita / PS4 / PS3", "")
<ipython-input-22-4559ff269911>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_noSub_no_DLC['clean_title'] = df_raw_noSub_no_DLC['clean_title'].str.replace(" - PS Vita", "")
<ipython-input-22-4559ff269911>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

### Clean DF

In [23]:
#New frame with cleaned data and only columns needed
Df_clean = df_raw_noSub_no_DLC[['customer_id', 'clean_title', 'star_rating']]
#reseting index
Df_clean.reset_index(drop=True, inplace=True)

print(f'Total number of customers in dataset:{Df_clean.customer_id.nunique()}')
print(f'Total number of games in dataset:{Df_clean.clean_title.nunique()}')

Total number of customers in dataset:76941
Total number of games in dataset:6001


In [24]:
# Subsetting to remove the customers with less than 2 reviews
Df_clean_subset = Df_clean[Df_clean['customer_id'].map(Df_clean['customer_id'].value_counts()) > 1]

print(f'Total number of customers in dataset:{Df_clean_subset.customer_id.nunique()}')
print(f'Total number of games in dataset:{Df_clean_subset.clean_title.nunique()}')
print((f'Total number of ratings in dataset:{len(Df_clean_subset)}'))

Total number of customers in dataset:11133
Total number of games in dataset:4756
Total number of ratings in dataset:35362


In [25]:
#Saving as new file
Df_clean_subset.to_csv('./data/GameRatings.csv', index=False)
Ratings = pd.read_csv('./data/GameRatings.csv')

### Preparing for modeling

In [26]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [27]:
reader = Reader(line_format='user item rating',
                sep=',', 
                rating_scale=(1, 5))

data = Dataset.load_from_df(Ratings[['customer_id', 'clean_title', 'star_rating']], reader=reader)

In [28]:
trainset, testset = train_test_split(data, test_size=0.2)

print('Number of users: ', trainset.n_users)
print('Number of items: ', trainset.n_items)
print('Number of ratings: ', trainset.n_ratings)

Number of users:  10829
Number of items:  4424
Number of ratings:  28289


In [29]:
#for looking back into de meaning of each id
trainset_iids = list(trainset.all_items())
iid_converter = lambda x: trainset.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

# Modeling

I will be using scikit-surprise models to built the recomender system
Using collaborative-filtering aproach, i will first train a memory-based model.
later i will iterate and perform gridsearch using model-based models

### KNN Baseline Model

In [31]:
import surprise
from surprise.prediction_algorithms import *

KNNbaseline_model = KNNBaseline()

KNNbaseline_model.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


**Testing baseline model**

In [44]:
from surprise import accuracy

#getting predictions using testset
KNNb_predictions = KNNbaseline_model.test(testset)

# Root Mean Square Error and Mean Absolute Errors
accuracy.rmse(KNNb_predictions)
accuracy.mae(KNNb_predictions)

RMSE: 1.3764
MAE:  1.0816


''

Prediction fails for 1.3 stars

In [48]:
from surprise.model_selection import cross_validate

KNNb_cv = cross_validate(algo = KNNbaseline_model, 
                         data = data, 
                         measures=['RMSE'], 
                         cv=10, 
                         n_jobs= -2,
                         return_train_measures=True)

In [49]:
KNNb_cv['test_rmse'].mean()

1.3770425758730576

from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

## Evaluation

## Deployment

## metadata

!ls data


import gzip

vec_tfidf

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('./data/meta_Video_Games.json.gz')